In [1]:
sys.path.append('../scripts/')
# %load ../scripts/notebook_setup.py
from notebook_setup import *
%matplotlib inline

<Logger ism3d (DEBUG)>
[<StreamHandler stderr (INFO)>]

################################################################################
Python version:   3.8.3 (default, May 16 2020, 10:05:49) 
[Clang 11.0.0 (clang-1100.0.33.17)]
Host Name:        Ruis-Mac-mini.local
Num of Core:      12
Total Memory:     32.0 GB
Available Memory: 10.84 GB
################################################################################
astropy            >=4.0        4.0         
emcee              >=3.0.0      3.0.3.dev4+gc14b212
corner             >=2.0        2.0.1       
tqdm               unspecified  4.43.0      
lmfit              unspecified  1.0.0       
asteval            >=0.9.14     0.9.18      
numexpr            >=2.6.9      2.7.1       
hickle             unspecified  4.0.0       
pyfftw             unspecified  0.12.0      
alpy               unspecified  1.0.0       
regions            unspecified  0.5.dev1001 
scipy              unspecified  1.5.0       
reproject          unspecifie


0.3.dev1
rx.astro@gmail.com

working dir: /Users/Rui/Resilio/Workspace/projects/ism3d/models/notebook



+ mockup1:  based on the VLA GN20 observation 1-channel
+ mockup2:  based on the ALMA G09 observation 1-channel
+ mockup3:  based on the ALMA G09 observation 240-channel

READ DATA

In [ ]:
model_name='mockup3'
outdir='../output/mockup/'

# option 1
#uvdata={}
#read_ms(vis=outdir+model_name+'_basis.ms',dataset=uvdata,keyrule='basename')

# option 2
uvdata=read_ms(vis=outdir+model_name+'_basis.ms')

# save to / reterive from .h5
to_hdf5(uvdata,outname=outdir+model_name+'_basis.h5',checkname=False,compression='gzip')
uvdata=from_hdf5(outdir+model_name+'_basis.h5')


READing: ../output/mockup/mockup3_basis.ms

------------------------------------------------------------------------------------------
data            complex64       (1154327, 240)         2 GiB             
uvw             float32         (1154327, 3)          13 MiB             
weight          float32         (1154327,)             4 MiB             
                ptile:          0%                   25.065651           
                ptile:          16%                  38.313652           
                ptile:          50%                  44.756321           
                ptile:          84%                  51.203125           
                ptile:          100%                 69.013229           
flag            bool            (1154327, 240)       264 MiB             
chanfreq        (240,)        93.0698 GHz    94.9369 GHz
chanwidth       (240,)        -7.8121 MHz    -7.8121 MHz
chanchi2        (240,)      2107.2754 3549150.0000
ndata            275884819
sumwt 

# Invert 

We perform imaging operations on the same visibility dataset using three different approachs and compare their precisions:

+ ism3d.uvhelper.inver
+ casatasks.tclean
+ galario

In [3]:
header=advise_header(uvdata['uvw'],
                    uvdata['phasecenter'],
                    uvdata['chanfreq'],
                    uvdata['chanwidth'],
                    antsize=12*u.m)

uu=uvdata['uvw'][:,0]
vv=uvdata['uvw'][:,1]
wv=uvdata['chanfreq'].to(u.m,equivalencies=u.spectral()).value
vis=uvdata['data']
wt=uvdata['weight']
flag=uvdata['flag']

cell=header['CDELT2']<<u.deg
imsize=header['NAXIS1']

print(imsize,cell.to(u.arcsec))

invert(vis=outdir+model_name+'_basis.ms',
       imagename=outdir+model_name+'_basis.images/casa',
       weighting='natural',
       cell=cell.to_value(u.arcsec),imsize=[imsize,imsize],onlydm=False)

cube=invert_ft(uu,vv,wv,vis,cell,imsize,wt=wt,flag=flag)
psf=make_psf(uu,vv,wv,cell,imsize,wt=wt,flag=flag)

fits.writeto(outdir+model_name+'_basis.images/nufft.image.fits',cube.T,header,overwrite=True)
fits.writeto(outdir+model_name+'_basis.images/nufft.psf.fits',psf.T,header,overwrite=True)

for version in ['image','psf']:
    tcube,thdr=fits.getdata(outdir+model_name+'_basis.images/casa.'+version+'.fits',header=True)
    cube,hdr=fits.getdata(outdir+model_name+'_basis.images/nufft.'+version+'.fits',header=True)
    cube_diff=np.squeeze(cube)-np.squeeze(tcube)
    fits.writeto(outdir+model_name+'_basis.images/diff.'+version+'.fits',cube_diff,thdr,overwrite=True)


420 0.3278515550731468 arcsec


2020-07-01 16:46:39	WARN	SynthesisParamsImage::buildCoordinateSystem (file casa-source/code/synthesis/ImagerObjects/SynthesisUtilMethods.cc, line 2124)	No rest frequency info, using the center of the selected spw(s):4.5656e+10 Hz. Velocity labelling may not be correct.
2020-07-01 16:46:40	WARN	task_tclean::SIImageStore::restore (file casa-source/code/synthesis/ImagerObjects/SIImageStore.cc, line 2089)	Restoring with an empty model image. Only residuals will be processed to form the output restored image.


(77108,) (77108,) (77108,) (77108,)
(77108,) (77108,) (77108,) (77108,)
